In [1]:
import pandas as pd
from datetime import datetime, timedelta, date
import matplotlib.pylab as plt
import yfinance as yf
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from numba import jit, prange
from pandas.plotting import register_matplotlib_converters
from sklearn import preprocessing

register_matplotlib_converters()

In [2]:
nvidia_df = pd.read_csv("../00_data/Sentiment_Only_Data_22MAR.gz", compression="gzip")

In [3]:
X_feautures_pre_merge = nvidia_df.copy()

In [4]:
X_feautures_pre_merge["Date"] = pd.to_datetime(
    X_feautures_pre_merge["Date"], format="%Y-%m-%d", utc=True
).dt.date

In [5]:
X_feautures_pre_merge.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Index', 'videoID', 'datePub',
       'searchedDate', 'VideoTitle', 'channelTitle', 'viewCount', 'likeCount',
       'dislikeCount', 'captionString', 'videoTitleSentimentScore_pos',
       'videoTitleSentimentScore_neg', 'captionSentimentScore_pos',
       'captionSentimentScore_neg', 'captionSentimentScore_com',
       'videoTitleSentimentScore_com', 'num_days_since_pub',
       'viewcounts_weighted', 'like_count_weighted', 'dislike_count_weighted',
       'like_dislike_ratio', 'weighted_like_dislike_ratio',
       'like_dislike_ratio_by_viewcounts_weighted', 'Date'],
      dtype='object')

In [6]:
X_feautures_pre_merge_1 = X_feautures_pre_merge.drop(['Unnamed: 0', 'Unnamed: 0.1', "Index"], axis =1).copy()

In [7]:
X_feautures_pre_merge_1["searchedDate"] = pd.to_datetime(
    X_feautures_pre_merge["searchedDate"], format="%Y-%m-%d %H:%M", utc=True
)

In [8]:
X_feautures_pre_merge_1

,videoID,datePub,searchedDate,VideoTitle,channelTitle,viewCount,likeCount,dislikeCount,captionString,videoTitleSentimentScore_pos,...,captionSentimentScore_com,videoTitleSentimentScore_com,num_days_since_pub,viewcounts_weighted,like_count_weighted,dislike_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted,Date
0,y4fb_R5Ogw0,2020-10-01 05:16:02+00:00,2020-10-02 01:45:21.553469+00:00,"2x NVIDIA RTX 3090 SLI Benchmarks: 500FPS, 700...",Gamers Nexus,367472,15701,260,so this setup that you're seeing is what we ha...,0.214,...,0.9997,0.2023,1.0,367472.000000,15701.000000,260.000000,0.983710,0.983710,361485.989098,2020-10-02
1,ALEXVtnNEwA,2020-09-01 17:15:57+00:00,2020-10-02 01:45:22.351801+00:00,NVIDIA GeForce Special Event Livestream,IGN,543115,16860,411,b welcome to my kitchen i hope all of you are ...,0.403,...,1.0000,0.4019,31.0,17519.838710,543.870968,13.258065,0.976203,0.976203,17102.917066,2020-10-02
2,E98hC9e__Xs,2020-09-01 16:42:07+00:00,2020-10-02 01:45:23.252012+00:00,NVIDIA GeForce RTX 30 Series | Official Launch...,NVIDIA GeForce,1484075,54741,1135,Welcome to my kitchen. I hope all of you are...,0.000,...,1.0000,0.0000,31.0,47873.387097,1765.838710,36.612903,0.979687,0.979687,46900.942857,2020-10-02
3,nMns8t4OhI0,2020-09-30 23:00:06+00:00,2020-10-02 01:45:23.922084+00:00,NVIDIA Responds to RTX 3080 Crashes...,JayzTwoCents,561238,27766,885,NaN,0.000,...,0.0000,0.0000,2.0,280619.000000,13883.000000,442.500000,0.969111,0.969111,271950.966947,2020-10-02
4,AG_ZHi3tuyk,2020-09-16 12:59:59+00:00,2020-10-02 01:45:24.592244+00:00,"Nvidia, you PROMISED! - RTX 3080 Review",Linus Tech Tips,3391565,141852,3961,- The RTX 3080 is the most powerful GPU we've ...,0.413,...,0.9997,0.5461,16.0,211972.812500,8865.750000,247.562500,0.972835,0.972835,206214.585797,2020-10-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20788,AG_ZHi3tuyk,2020-09-16 12:59:59+00:00,2021-03-19 18:03:20.670031+00:00,"Nvidia, you PROMISED! - RTX 3080 Review",Linus Tech Tips,3741789,148430,4216,- The RTX 3080 is the most powerful GPU we've ...,0.413,...,0.9997,0.5461,185.0,20225.886486,802.324324,22.789189,0.972381,0.972381,19667.258436,2021-03-19
20789,3dJhDqLjPN0,2021-02-24 21:59:34+00:00,2021-03-19 18:03:21.320922+00:00,NVIDIA (NVDA) Stock Analysis - Q4 Earnings + I...,The Investor Channel,7773,269,1,what is going on investors hopefully guys had ...,0.167,...,0.9986,0.3400,23.0,337.956522,11.695652,0.043478,0.996296,0.996296,336.704831,2021-03-19
20790,H1DApIvOCMw,2021-02-26 17:35:37+00:00,2021-03-19 18:03:21.788746+00:00,My SALTY Review of the NVIDIA RTX 3060...,JayzTwoCents,1013948,51773,702,with no parts markup and only a 75 build fee r...,0.000,...,0.9939,0.0000,22.0,46088.545455,2353.318182,31.909091,0.986622,0.986622,45471.982159,2021-03-19
20791,Y5J2bgXiK9g,2021-03-09 17:00:10+00:00,2021-03-19 18:03:22.387116+00:00,Should Nvidia Stop Selling 3060s?,LMG Clips,53940,2238,33,uh the you know the 30 80 eth mining wait isn'...,0.000,...,0.9982,-0.2960,11.0,4903.636364,203.454545,3.000000,0.985469,0.985469,4832.381410,2021-03-19


In [9]:
type(X_feautures_pre_merge_1["searchedDate"][0])

pandas._libs.tslibs.timestamps.Timestamp

In [10]:
pre_nan_drop_X_feautures_pre_merge = X_feautures_pre_merge_1.groupby(pd.Grouper(key="searchedDate", freq='H', offset = "30min")).mean()

In [11]:
pre_nan_drop_X_feautures_pre_merge 

,viewCount,likeCount,dislikeCount,videoTitleSentimentScore_pos,videoTitleSentimentScore_neg,captionSentimentScore_pos,captionSentimentScore_neg,captionSentimentScore_com,videoTitleSentimentScore_com,num_days_since_pub,viewcounts_weighted,like_count_weighted,dislike_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted
searchedDate,,,,,,,,,,,,,,,,
2020-10-02 01:30:00+00:00,1750507.60,69801.60,1799.70,0.10300,0.07330,0.10010,0.03720,0.748180,0.032240,16.60,137832.492033,5912.089949,150.234402,0.973936,0.973936,134451.295653
2020-10-02 02:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02 03:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02 04:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02 05:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-19 13:30:00+00:00,938884.40,52084.56,1250.92,0.04094,0.06668,0.11092,0.03736,0.839152,-0.003934,112.68,14804.447178,1054.840931,23.871143,0.954151,0.954151,14442.003775
2021-03-19 14:30:00+00:00,940417.64,52157.80,1252.04,0.03808,0.06668,0.10980,0.03648,0.839128,-0.005172,114.72,14681.735511,1025.946601,23.450953,0.954274,0.954274,14321.077933
2021-03-19 15:30:00+00:00,879356.68,46399.36,1048.00,0.03808,0.06496,0.10348,0.03344,0.799200,-0.004168,108.48,14717.656240,1028.703032,21.186843,0.958955,0.958955,14376.975594


In [14]:
post_nan_drop_X_feautures_post_merge = pre_nan_drop_X_feautures_pre_merge [pre_nan_drop_X_feautures_pre_merge ['viewCount'].notna()]

In [16]:
post_nan_drop_X_feautures_post_merge.sample(25)

,viewCount,likeCount,dislikeCount,videoTitleSentimentScore_pos,videoTitleSentimentScore_neg,captionSentimentScore_pos,captionSentimentScore_neg,captionSentimentScore_com,videoTitleSentimentScore_com,num_days_since_pub,viewcounts_weighted,like_count_weighted,dislike_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted
searchedDate,,,,,,,,,,,,,,,,
2020-12-24 13:30:00+00:00,1314751.88,58454.10,1318.48,0.05544,0.09708,0.11308,0.03628,0.839172,-0.041980,92.08,34280.549415,1996.649177,39.894193,0.969127,0.969127,33568.814977
2021-01-15 13:30:00+00:00,1214078.84,53602.70,1276.32,0.05172,0.07608,0.09120,0.02928,0.718940,-0.029464,105.36,29554.908928,1369.072080,46.862301,0.958797,0.958797,28435.243294
2021-02-01 13:30:00+00:00,1238969.94,54315.52,1295.62,0.03264,0.11668,0.10196,0.03156,0.799016,-0.098672,107.00,18493.077239,893.675294,23.973884,0.958605,0.958605,17964.492133
2020-11-21 13:30:00+00:00,1151365.88,47561.44,1073.76,0.06872,0.05844,0.11772,0.03140,0.877256,-0.010128,79.64,26741.822159,1179.782258,28.897347,0.963015,0.963015,26096.544254
2020-11-06 14:30:00+00:00,11871.04,319.32,9.16,0.07284,0.00000,0.11908,0.02544,0.879068,0.131832,57.80,264.413463,12.458646,0.272114,0.972329,0.972329,257.498211
2020-11-26 20:30:00+00:00,1135327.28,46233.96,1098.28,0.06872,0.07604,0.10132,0.02468,0.757080,-0.031484,86.20,22353.510501,994.750924,25.542751,0.960092,0.960092,21787.012773
2021-02-16 13:30:00+00:00,1193175.46,52646.48,1220.64,0.03264,0.04536,0.10356,0.03220,0.799312,-0.027680,127.64,23158.760729,1514.474491,27.171267,0.959405,0.959405,22675.489863
2020-11-23 16:30:00+00:00,1197889.00,49718.64,1139.76,0.06872,0.04728,0.11048,0.03000,0.836852,0.008884,85.16,25135.852086,1090.593886,26.402308,0.962979,0.962979,24528.095390
2021-02-16 20:30:00+00:00,1091426.12,49695.28,1150.16,0.05140,0.06976,0.10304,0.03336,0.799332,-0.018068,104.64,28994.554069,1889.717202,32.271156,0.960396,0.960396,28431.213951


In [17]:
post_nan_drop_X_feautures_post_merge.to_csv("../00_data/By_hour_feature_aggrigated_04APR.gz", compression = "gzip")